<a href="https://colab.research.google.com/github/Varshap44/CHATBOT/blob/main/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers langchain gradio
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found ex

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextIteratorStreamer
from threading import Thread

def initialize_model_and_tokenizer(model_name="EleutherAI/gpt-neo-1.3B"):
    """
    Initialize the model and tokenizer using a pre-trained model from Hugging Face.

    Parameters:
    model_name (str): The name of the pre-trained model to load.

    Returns:
    model: The loaded language model.
    tokenizer: The tokenizer associated with the model.
    """
    model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

# Initialize the model and tokenizer
model, tokenizer = initialize_model_and_tokenizer()

# Create the Hugging Face pipeline with max_new_tokens
pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [18]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

def initialize_langchain(pipeline):
    """
    Initialize LangChain with the given pipeline.

    Parameters:
    pipeline: The Hugging Face pipeline.

    Returns:
    chain: The initialized LangChain.
    """
    llm = HuggingFacePipeline(pipeline=pipeline)
    prompt = PromptTemplate(input_variables=["input"], template="{input}")
    chain = LLMChain(prompt=prompt, llm=llm)
    return chain

# Initialize LangChain
langchain_chain = initialize_langchain(pipeline)

In [19]:
from langchain import PromptTemplate

# Define a template for data science queries
template = """Question: {question}".
Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

In [20]:
import re

# Function to check for complete response
def extract_complete_response(response, question):
    """
    Extract the complete response based on the question.

    Parameters:
    response (str): The generated response from the model.
    question (str): The question asked to the model.

    Returns:
    str: The extracted complete response.
    """
    stop_phrases = [
        "In conclusion", "Therefore", "Lastly", "Finally", "That's all", "That is all", "That's it", "That is it"
    ]

    for phrase in stop_phrases:
        if phrase in response:
            return response.split(phrase)[0] + phrase

    return response

# Create an LLM chain using the LangChain chain and the prompt template
langchain_chain.verbose = True

# Use the correct input key for this chain
question = "Explain solid principles in programming using python."
response = langchain_chain.run({"input": question})

# Extract the text from the HuggingFacePipeline's output if it's not a string
if not isinstance(response, str):
    response = response[0]['generated_text']  # Assuming the text is in the 'generated_text' key of the first element

# Extract complete response
response = extract_complete_response(response, question)
print(response)




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Explain solid principles in programming using python.

> Finished chain.
Explain solid principles in programming using python.

This is a continuation of a previous question that I answered here.
You can find my python answer here (You can skip my section if you like):
The code can be seen in action on:
https://gist.github.com/kashifj/779925
I am not planning to update it as I don't feel it is necessary, nor do I want to create a new answer.

A:

For starters, the "python" line doesn't say Python here, it's "python3".

To quote my original question: You can find my Python code here (You can skip my section if you like):

I guess someone else will edit this answer in the future when new and updated versions of Python are released. For now, the point is that it's python3. The answer can still be written in python2, it just won't have the "Python" link.
In general, we


In [21]:
import gradio as gr

# Function to handle user input and generate responses
def user(user_message, history):
    """
    Process the user input and update the conversation history.

    Parameters:
    user_message (str): The message input by the user.
    history (list): The conversation history.

    Returns:
    tuple: Updated user input and conversation history.
    """
    return "", history + [[user_message, None]]

In [22]:
def bot(history):
    """
    Generate a response using the LLM chain and update the conversation history.

    Parameters:
    history (list): The conversation history.

    Returns:
    list: Updated conversation history with the bot's response.
    """
    print("Question: ", history[-1][0])
    bot_message = langchain_chain.run({"input": history[-1][0]})
    if not isinstance(bot_message, str):
        bot_message = bot_message[0]['generated_text']  # Assuming the text is in the 'generated_text' key of the first element

    # Extract complete response
    bot_message = extract_complete_response(bot_message, history[-1][0])
    print("Response: ", bot_message)
    history[-1][1] = bot_message
    return history

In [23]:
# Create the Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Define the interactions
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(bot, chatbot, chatbot)
    clear.click(lambda: None, None, chatbot, queue=False)

# Launch the Gradio interface
demo.queue()
demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1859b8a0500e5194a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
